In [ ]:
# ==========================================
# Invoice PDF extractor (Azure Document Intelligence)

# --- Imports principales ---
import os
import pandas as pd
import re

# --- Azure SDK: credenciales---
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature

# --- Azure endpoint + Key1 ---
endpoint = "https://eastus.api.cognitive.microsoft.com/"
key = "256c36c92ab74418bdb2642d938fc263"

def _norm(s: str) -> str:
    # Normaliza texto (minúsculas y espacios) para comparar keys de KV pairs
    return re.sub(r"\s+", " ", (s or "").strip()).lower()

def _kv_to_dict(kv_pairs):
    # Convierte key_value_pairs del modelo a diccionario {key_normalizada: value}
    out = {}
    if not kv_pairs:
        return out
    for kv in kv_pairs:
        k = getattr(kv, "key", None)
        v = getattr(kv, "value", None)
        ktxt = (getattr(k, "content", "") or "").strip()
        vtxt = (getattr(v, "content", "") or "").strip()
        if ktxt and vtxt:
            out[_norm(ktxt)] = vtxt
    return out

def extract_material_only_by_label(result):
    # Material: primero intenta Key-Value (si el modelo lo detecta), si no usa regex del OCR
    content = getattr(result, "content", "") or ""

    kv_map = _kv_to_dict(getattr(result, "key_value_pairs", None))
    for k, v in kv_map.items():
        if ("your article" in k or "your articleno" in k) and "no" in k:
            return v.strip() if v else None

    # Regex tolerante a variaciones de OCR (misma línea o línea siguiente)
    patterns = [
        r"your\s*article\s*no\.?\s*[: ]\s*([A-Z0-9\-\/]+)",
        r"your\s*articleno\.?\s*[: ]\s*([A-Z0-9\-\/]+)",
        r"your\s*article\s*no\.?\s*[: ]?\s*[\r\n]+\s*([A-Z0-9\-\/]+)",
        r"your\s*articleno\.?\s*[: ]?\s*[\r\n]+\s*([A-Z0-9\-\/]+)",
    ]
    for p in patterns:
        m = re.search(p, content, flags=re.IGNORECASE)
        if m:
            return m.group(1).strip()
    return None

def extract_total_gross_weight(content: str):
    # Total gross weight: se extrae desde OCR (normalmente no viene como field estructurado)
    m = re.search(
        r"total\s+gross\s+weight\s*[:=]?\s*([0-9]{1,3}(?:[.,][0-9]{3})*(?:[.,][0-9]+)?)\s*kg",
        content,
        flags=re.IGNORECASE
    )
    return (m.group(1).strip() + " kg") if m else None

def fallback_quantity_from_content(content: str):
    # Fallback: si no detecta una tabla, toma el primer número seguido de "pieces"
    m = re.search(
        r"([0-9]{1,3}(?:[.,][0-9]{3})*(?:[.,][0-9]+)?)\s*pieces\b",
        content,
        flags=re.IGNORECASE
    )
    return (m.group(1).strip() + " pieces") if m else None

def extract_quantity_from_tables(result, material: str | None = None):
    # Quantity: se toma desde tablas detectadas por Layout (columna 'Quantity')
    tables = getattr(result, "tables", None) or []
    if not tables:
        return None

    for t in tables:
        grid = {}
        for cell in (t.cells or []):
            grid[(cell.row_index, cell.column_index)] = (cell.content or "").strip()

        header_row = None
        qty_col = None

        # Identifica la fila header donde aparece "quantity"
        for r in range(t.row_count):
            row = [grid.get((r, c), "") for c in range(t.column_count)]
            norm_row = [_norm(x) for x in row]
            if any("quantity" in x for x in norm_row):
                header_row = r
                for c, val in enumerate(norm_row):
                    if "quantity" in val:
                        qty_col = c
                        break
                break

        if header_row is None or qty_col is None:
            continue

        # Si el material está en la tabla, intenta tomar la Quantity de esa fila
        if material:
            for r in range(header_row + 1, t.row_count):
                row = [grid.get((r, c), "") for c in range(t.column_count)]
                if any(material in x for x in row):
                    q = grid.get((r, qty_col), "")
                    return q.strip() if q else None

        # Si no, toma la primera fila de datos con número
        for r in range(header_row + 1, t.row_count):
            q = grid.get((r, qty_col), "")
            if q and re.search(r"\d", q):
                return q.strip()

    return None

def analyze_invoice():
    # --- Aqui vas a poner el URL de los PDFs a analizar---
    invoicePath = r"C:\Users\z004zfhd\OneDrive - Siemens AG\Documentos\Business Analytics - Machine Learning\Invoice 202410882.pdf"
    if not os.path.exists(invoicePath):
        raise FileNotFoundError(invoicePath)

    
    client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    # --- Analiza con prebuilt-layout ---
    with open(invoicePath, "rb") as f:
        poller = client.begin_analyze_document(
            model_id="prebuilt-layout",
            analyze_request=f,
            content_type="application/pdf",
            features=[DocumentAnalysisFeature.KEY_VALUE_PAIRS],
        )

    result = poller.result()
    content = getattr(result, "content", "") or ""

    # --- Extracción de campos ---
    material = extract_material_only_by_label(result)
    quantity = extract_quantity_from_tables(result, material=material) or fallback_quantity_from_content(content)
    total_gross_weight = extract_total_gross_weight(content)

    # --- DataFrame + Excel ---
    df = pd.DataFrame([{
        "Material": material,
        "Quantity": quantity,
        "Total Gross Weight": total_gross_weight
    }])

    output_path = r"C:\Users\z004zfhd\OneDrive - Siemens AG\Documentos\Business Analytics - Machine Learning\invoice_extraction.xlsx"
    df.to_excel(output_path, index=False)

    print("Excel guardado en:", output_path)
    print(df)

if __name__ == "__main__":
    analyze_invoice()



  


Excel guardado en: C:\Users\z004zfhd\OneDrive - Siemens AG\Documentos\Business Analytics - Machine Learning\invoice_extraction.xlsx
         Material   Quantity Total Gross Weight
0  A5E02291075030  12.000,00          303,20 kg


In [ ]:
# ==========================================
# Azure Document Intelligence - Extract (DN PDF):
# Extrae: Item No, Quantity, Gross weight

import os
import pandas as pd
import re

# Azure SDK
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature

# Endpoint y API Key
endpoint = "https://eastus.api.cognitive.microsoft.com/"
key = "256c36c92ab74418bdb2642d938fc263"


def _norm(s: str) -> str:
    # Normaliza texto para comparar headers de tablas
    return re.sub(r"\s+", " ", (s or "").strip()).lower()


def extract_gross_weight(content: str):
    """
    Extrae Gross Weight desde el OCR del documento
    """
    if not content:
        return None

    m = re.search(
        r"gross\s*weight\s*[:=]?\s*([0-9]{1,3}(?:[.,][0-9]{3})*(?:[.,][0-9]+)?)\s*kg",
        content,
        flags=re.IGNORECASE
    )
    return (m.group(1).strip() + " kg") if m else None


def extract_itemno_quantity_from_tables(result):
    """
    Extrae Item No y Quantity desde tablas detectadas por prebuilt-layout.
    - Detecta header con 'Item No' y 'Quantity'
    - Toma la primera fila de datos debajo del header
    """
    tables = getattr(result, "tables", None) or []
    if not tables:
        return None, None

    for t in tables:

        # Construye matriz de celdas detectadas
        grid = {}
        for cell in (t.cells or []):
            grid[(cell.row_index, cell.column_index)] = (cell.content or "").strip()

        header_row = None
        qty_col = None
        item_col = None

        # Busca la fila header que contenga Item No y Quantity
        for r in range(t.row_count):
            row = [grid.get((r, c), "") for c in range(t.column_count)]
            nrow = [_norm(x) for x in row]

            has_qty = any("quantity" in x for x in nrow)
            has_item = any(("item" in x and "no" in x) for x in nrow)

            if has_qty and has_item:
                header_row = r
                for c, val in enumerate(nrow):
                    if qty_col is None and "quantity" in val:
                        qty_col = c
                    if item_col is None and ("item" in val and "no" in val):
                        item_col = c
                break

        if header_row is None or qty_col is None or item_col is None:
            continue

        # Extrae la primera fila de datos debajo del header
        for r in range(header_row + 1, t.row_count):
            qty = grid.get((r, qty_col), "").strip()
            item_no = grid.get((r, item_col), "").strip()

            if qty or item_no:
                return (item_no if item_no else None), (qty if qty else None)

    return None, None


def analyze_dn():

    dnPath = r"C:\Users\z004zfhd\OneDrive - Siemens AG\Documentos\Business Analytics - Machine Learning\DN 45759.pdf"

    # Validación existencia archivo
    if not os.path.exists(dnPath):
        raise FileNotFoundError(dnPath)

    # Inicializa cliente Azure Document Intelligence
    client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    # Analiza documento usando modelo Layout (tablas + texto)
    with open(dnPath, "rb") as f:
        poller = client.begin_analyze_document(
            model_id="prebuilt-layout",
            analyze_request=f,
            content_type="application/pdf",
            features=[DocumentAnalysisFeature.KEY_VALUE_PAIRS],
        )

    result = poller.result()
    content = getattr(result, "content", "") or ""

    # 1) Item No y Quantity desde tablas
    item_no, quantity = extract_itemno_quantity_from_tables(result)

    # 2) Gross weight desde texto OCR
    gross_weight = extract_gross_weight(content)

    # Construcción DataFrame
    data = {
        "Item No": [item_no],
        "Quantity": [quantity],
        "Gross weight": [gross_weight]
    }

    df = pd.DataFrame(data)

    # Muestra resultado
    print(df)


if __name__ == "__main__":
    analyze_dn()


          Item No    Quantity Gross weight
0  L1V30361994011  20.800 pcs  1.300,00 kg
